## Exploring the data

So let's learn how we set up the data. 

In [1]:
# default_exp utils

In [2]:
#hide
import os,sys

In [3]:
#export
root = "D:/data_sets/24_garden"

In [4]:
#export
import tensorflow as tf

In order to parse the data, we need to "unpack" each sample to bring it into a `Dataset`. `tfrecord` has two special functions we can use, `unpack_int_64_list` and `unpack_bytes_list` (our classes are integers and our images are bytes)

In [5]:
#export
def unpack_int64_list(feature):
    return feature.int64_list.value

In [6]:
#export
def unpack_bytes_list(feature):
    return feature.bytes_list.value

In [7]:
#export
def unpack_sample(feats):
    return {
        'class' : unpack_int64_list(feats['class']),
        'image' : unpack_bytes_list(feats['image'])
    }

In [8]:
#export
class Reader:
    def __init__(self, fname, unpack_sample, compression=None):
        self._engine = iter(tf.compat.v1.io.tf_record_iterator(
            fname, compression_code(compression)))
        self._unpack_sample = unpack_sample

    def __enter__(self):
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        pass

    def __iter__(self):
        return self

    def __next__(self):
        buffer = next(self._engine)
        example = tf.train.Example()
        example.ParseFromString(buffer)
        return self._unpack_sample(example.features.feature)

    def read_sample(self):
        try:
            return __next__(self)
        except StopIteration:
            return None

In [9]:
#export
def compression_code(compression):
    if compression is None:
        return None
    code = _tf_compression_revmap.get(compression)
    if code is None:
        raise ValueError(
            'Unknown or unsupported compression type: ' + compression)